# Algorithm: Lloyd's K-means Clustering

__Initialize__: Dataset $\mathcal{D} = \{\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{n} \in \mathbb{R}^{m}\}$, number of clusters $K \in \mathbb{Z}^{+}$, maximum iterations $\texttt{maxiter} \in \mathbb{Z}^{+}$, and initial centroids $\{\boldsymbol{\mu}_{1}, \boldsymbol{\mu}_{2}, \ldots, \boldsymbol{\mu}_{K} \in \mathbb{R}^{m}\}$. Set convergence flag $\texttt{converged} \leftarrow \texttt{false}$ and iteration counter $\texttt{iter} \leftarrow 0$.

__Output__: Cluster assignments for each point (where point $i$ is assigned to cluster $c_{i} \in \{1,\ldots,K\}$) and updated cluster centroids $\{\boldsymbol{\mu}_{1}, \boldsymbol{\mu}_{2}, \ldots, \boldsymbol{\mu}_{K}\}$.

> **Initialization strategies**: Random initialization can lead to poor local optima. Better initialization approaches include:
> 
> * **K-means++**: Select initial centroids using probabilistic sampling where each point's selection probability is proportional to its squared distance from the nearest already-chosen centroid. This spreads centroids apart and improves convergence quality.
> * **Multiple random starts**: Run the algorithm 10–50 times with different random initializations and select the result with the smallest objective function value $J$. Most implementations use 10 as default.
> * **Convergence tolerance**: Set $\epsilon = 10^{-4}$ (standard default) to $10^{-6}$ for tighter convergence. Smaller values require more iterations.
> * **Maximum iterations**: Use $T = 300$ (standard default) for most applications, or scale with the number of clusters using $T \sim 100 + 10K$. K-means typically converges in 10–50 iterations, but setting higher $T$ provides a safety margin against non-convergence.

While $\texttt{converged}$ is $\texttt{false}$ and $\texttt{iter} < \texttt{maxiter}$ __do__:
1. **Assignment step**: For each data point $\mathbf{x}_{i} \in \mathcal{D}$ (where $i = 1, \ldots, n$), assign it to the nearest cluster centroid:
   $$c_{i} \leftarrow \arg\min_{j} \underbrace{\lVert\mathbf{x}_{i} - \boldsymbol{\mu}_{j}\rVert_{2}^{2}}_{=\;d(\mathbf{x}_{i}, \boldsymbol{\mu}_{j})^2}$$
    where $c_{i} \in \{1,\ldots,K\}$ is the cluster assignment for point $i$.
2. **Update step**: Store the current centroids $\hat{\boldsymbol{\mu}}_{j} \leftarrow \boldsymbol{\mu}_{j}$ for all $j$. Then, for each cluster $j = 1$ to $K$, recompute the centroid as the mean of all points assigned to cluster $j$:
   $$\boldsymbol{\mu}_{j} \leftarrow \frac{1}{|\mathcal{C}_{j}|} \sum_{\mathbf{x} \in \mathcal{C}_{j}} \mathbf{x}$$
   where $\mathcal{C}_{j} = \{\mathbf{x}_{i} : c_{i} = j\}$ is the set of points assigned to cluster $j$, and $|\mathcal{C}_{j}|$ is the number of points in that cluster.
3. **Convergence check**: Compute the maximum change in centroid positions:
   $$\Delta = \max_{j} \lVert\boldsymbol{\mu}_{j} - \hat{\boldsymbol{\mu}}_{j}\rVert_{2}^{2}$$
   If $\Delta < \epsilon$ (where $\epsilon$ is a small tolerance threshold), set $\texttt{converged} \leftarrow \texttt{true}$ and exit the loop. Otherwise, increment $\texttt{iter} \leftarrow \texttt{iter} + 1$ and continue to the next iteration.
    - Typical values for the convergence tolerance are $\epsilon \in \{10^{-4}, 10^{-6}\}$. Smaller values yield tighter convergence at the cost of more iterations. When features are standardized to unit variance, these default values work well; for unstandardized data, scale $\epsilon$ proportionally to the squared feature magnitudes.
    - If $\texttt{iter} \geq \texttt{maxiter}$, issue warning that maximum iterations reached without convergence and exit.

> __Practical considerations:__
>
> * __Convergence guarantee__: K-means is guaranteed to converge to a local minimum. The objective function $J = \sum_{i=1}^{n} \left\|\mathbf{x}_{i} - \boldsymbol{\mu}_{c_{i}}\right\|_{2}^{2}$ (where $\boldsymbol{\mu}_{c_{i}}$ denotes the centroid of the cluster that point $i$ is assigned to) decreases monotonically because: (1) the assignment step assigns each point to its nearest centroid, which cannot increase $J$, and (2) the update step computes centroids as means, which minimizes the sum of squared distances for the current assignments. Since there are finitely many possible clusterings and $J$ is non-increasing, convergence is guaranteed in finite iterations, typically within 10–50 iterations.
>
> * __Computational complexity__: The complexity is $O(nKm \cdot t)$, where $t$ is iterations, $n$ is data points, and $m$ is dimensionality. For large datasets (typically $n > 100{,}000$), mini-batch K-means reduces per-iteration cost to $O(Bm)$ where $B$ is the mini-batch size (commonly $B = 100$ to $1000$), trading some accuracy for speed. Mini-batch K-means randomly samples a subset of points for each iteration, making it practical for datasets too large to fit in memory.

___